<a href="https://colab.research.google.com/github/jenhuluck/NLP_projects/blob/main/sentiment_classification_using_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install ktrain

     |████████████████████████████████| 25.3MB 131kB/s 
     |████████████████████████████████| 6.8MB 40.0MB/s 
     |████████████████████████████████| 983kB 54.7MB/s 
     |████████████████████████████████| 266kB 55.7MB/s 
     |████████████████████████████████| 1.3MB 47.1MB/s 
     |████████████████████████████████| 1.2MB 49.1MB/s 
     |████████████████████████████████| 471kB 49.6MB/s 
     |████████████████████████████████| 890kB 48.5MB/s 
     |████████████████████████████████| 2.9MB 60.5MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.4-cp36-none-any.whl size=25276443 sha256=97c4d3c231f919df024e7f8bf995a045f3aeb349fce2d89802517c211c879d08
  Stored in directory: /root/.cache/pip/wheels/1b/77/8a/bdceaabc308e7178d575278bf6143b7d1a9b939a1e40c56b88
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=406fdc341ed3bb5f017dad81895ec9923eed470e288624030b23dad60e09ec29
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [3]:
import tensorflow as tf

In [4]:
tf.__version__

'2.4.1'

In [5]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [6]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text 

In [7]:
data_train = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx',dtype = str)
data_test = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx', dtype = str)


In [8]:
data_train.tail()

,Reviews,Sentiment
24995,Everyone plays their part pretty well in this ...,pos
24996,It happened with Assault on Prescient 13 in 20...,neg
24997,My God. This movie was awful. I can't complain...,neg
24998,"When I first popped in Happy Birthday to Me, I...",neg
24999,"So why does this show suck? Unfortunately, tha...",neg


In [9]:
data_test.shape, data_train.shape

((25000, 2), (25000, 2))

In [10]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=data_train, 
                                                                   text_column='Reviews',
                                                                   label_columns='Sentiment',
                                                                   val_df = data_test,
                                                                   maxlen = 500,
                                                                   preprocess_mode='bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [11]:
model = text.text_classifier(name='bert', train_data=(X_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 500
done.


In [12]:
learner = ktrain.get_learner(model=model, 
                   train_data=(X_train,y_train),
                   val_data = (X_test, y_test),
                   batch_size = 6
                   )

In [13]:
#find out best learning rate
#learner.lr_find()
#learner.lr_plot()
#Not recommend for the first time, since it may take days or many days to find out


In [14]:
learner.fit_onecycle(lr = 2e-5, epochs=1)



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - 5248s 1s/step - loss: 0.3316 - accuracy: 0.8488 - val_loss: 0.0992 - val_accuracy: 0.9711


In [15]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [16]:
test_data = ['This movie was horrible!', 'The film is really sucked.', 'Beautiful moive. Great plot.']

In [17]:
predictor.predict(test_data)

['neg', 'neg', 'pos']

In [18]:
predictor.save('/content/bert')

In [19]:
!zip -r /content/bert.zip /content/bert

  adding: content/bert/ (stored 0%)
  adding: content/bert/tf_model.h5 (deflated 11%)
  adding: content/bert/tf_model.preproc (deflated 52%)


In [20]:
predictor_load = ktrain.load_predictor('/content/bert')

In [21]:
predictor_load.predict(test_data)

['neg', 'neg', 'pos']